Question 1

In this assignment you will implement one or more algorithms for the all-pairs shortest-path problem.  Here are data files describing three graphs: g1.txt, g2.txt, g3.txt

The first line indicates the number of vertices and edges, respectively.  Each subsequent line describes an edge (the first two numbers are its tail and head, respectively) and its length (the third number).  NOTE: some of the edge lengths are negative.  NOTE: These graphs may or may not have negative-cost cycles.

Your task is to compute the "shortest shortest path".  Precisely, you must first identify which, if any, of the three graphs have no negative cycles.  For each such graph, you should compute all-pairs shortest paths and remember the smallest one (i.e., compute min of {u,v in V} d(u,v), where d(u,v) denotes the shortest-path distance from u to v). 

If each of the three graphs has a negative-cost cycle, then enter "NULL" in the box below.  If exactly one graph has no negative-cost cycles, then enter the length of its shortest shortest path in the box below.  If two or more of the graphs have no negative-cost cycles, then enter the smallest of the lengths of their shortest shortest paths in the box below.

In [1]:
import math
import heapq 

def load(filename):
    """
        To use for loading data to nodes (represent as a nested dictionary), in order to load to Graph class
        filename: input filename
        output: count of vertice, count of edges, dictionary of data with key as tail, and value as head and their length,
        another dictionary of data with key as head, and value as tail and their length
        ex. output: {1: {{2: 1}, {8: 2}}, 2: {{1: 1}, {3: 1}}}
    """
    output = {}
    output_reverse = {}
    
    with open(filename) as f:
        file = f.readlines()
        vertice_count, edge_count = file[0].strip().rsplit(" ")
        vertice_count = int(vertice_count)
        edge_count = int(edge_count)
        
        for i in range(1, len(file)):
            tail, head, length = file[i].strip().rsplit(" ")
            try:
                output[int(tail)][int(head)] = int(length)
            except:
                output[int(tail)] = {}
                output[int(tail)][int(head)] = int(length)
            
            try:
                output_reverse[int(head)][int(tail)] = int(length)
            except:
                output_reverse[int(head)] = {}
                output_reverse[int(head)][int(tail)] = int(length)
    
    return vertice_count, edge_count, output, output_reverse

    

class Graph:
    def __init__(self, nodes, reversed_nodes, node_count, edge_count):
        self.nodes = nodes
        self.reversed_nodes = reversed_nodes
        self.edge_count = edge_count
        self.node_count = node_count
        self.path = {}   # {("current", v): total_length} is the result of bellman_ford algorithm if no negative cost cycle
        self.shortest_path = {i:{} for i in range(1, node_count + 1)}   #to log the shortest path distance for each node
        self.distance = {i: math.inf for i in range(1, node_count + 1)}  #to log the distance in Dijkstra Algorithm
        self.checked = set()    # to be used in Dijkstra Algorithm
        self.heap = []      # to be used in Dijkstra Algorithm
        self.shortest = math.inf     #to log the shortest path of all-pair shortest path
            
            
    def bellman_ford(self):
        # add a new vertex 0 and a new edge (0, v) with length 0 for each v in Graph, and assume vertex 0 is the start point
        # therefore, edge(1, v) = 0 for each v in Graph
        # use (last, v) represent last round and (current, v) represents current round
        
        # initiate edge(1, v) = 0 for each v in Graph
        for v in range(1, self.node_count + 1):
            self.path[("last", v)] = 0
        
        # loop through i from 2 to total edge cout
        for i in range(2, self.edge_count + 1):
            # loop through all vertices
            for v in range(1, self.node_count + 1):
                # case 1: A[i - 1, v] (If path has <= (i - 1) edges, it is a shortest s-v path with (i - 1) edges.)
                last_v = self.path[("last", v)]
                
                # case 2: min(w, v) in E {A[i - 1, w] + c(wv)} (If path has i edges with last hop (w, v), then P' is a shortest
                # s-w path with <= (i - 1) edges.)
                last_w = math.inf
                new_dist = math.inf
                
                if v in self.reversed_nodes.keys():
                    for vertex, length in self.reversed_nodes[v].items():
                        last_w = self.path[("last", vertex)]
                        if new_dist > last_w + length:
                            new_dist = last_w + length
                
                # find the minimum length between case 1 and case 2
                self.path[("current", v)] = min(last_v, new_dist)
            
                # if not last comparison, copy the current run result as last run result
                if i != self.edge_count:
                    self.path[("last", v)] = self.path[("current", v)]
                
                # if last comparison, check if all last run result same as current run result. If not, it means there's
                # negative cost cycle
                else:
                    if self.path[("last", v)] != self.path[("current", v)]:
                        return False
                    
        return True
    
    
    def dijkstra(self, start):
        
        """
            Dijkstra Algorithm to calculate the shortest distance from start vertex to each other vertices in the Graph
        """
        self.heap = []
        self.checked.clear()
        vertex = start
        heapq.heappush(self.heap, (0, vertex))
        
        # as long as not all path has been visited
        while len(self.checked) < self.node_count:
            
            if len(self.heap) == 0:
                break
                
            # Visit the vertex that has smallest distance, meaning first element of heap (smallest Dijstra greedy score)
            distance, vertex = heapq.heappop(self.heap)
            
            # skip vertex if it's already visited, need to do this as this algorithm keeps adding update short distance to heap
            if vertex in self.checked:
                continue
            
            # As we visit each unvisited neighbor, we update their distance from the starting node to the smaller amount.
            # then add the (new_distance, vertex) tuple into heap
            if vertex in self.nodes.keys():
                for head, length in self.nodes[vertex].items():
                    if head not in self.checked:
                        # new distance of vertex = distance of parent vertex + the weight from parent vertex to this vertes
                        new_distance = distance + length     
                        
                        if self.distance[head] > new_distance:
                            heapq.heappush(self.heap, (new_distance, head))
                            self.distance[head] = new_distance
                    
                
            # After updating all neighbors, add the vertex to checked
            self.checked.add(vertex)

    
        
    def johnson(self):
        # Run Bellman-Ford on Graph  G' with source vertex 0, to calculate new vertex weight p; or notify negative cost cycle
        if not self.bellman_ford():
            return "There is Negative Cost Cycle!!"
            
        else:
            # For each v in G, define p(v) = length of a shortest s -> v path in G'. 
            # For each edge e = (u, v) in G, define c(e)' = c(e) + p(u) - p(v) .
            for vertex, values in self.nodes.items():
                for head, length in values.items():
                    self.nodes[vertex][head] = self.nodes[vertex][head] + self.path[("current", vertex)] - self.path[("current", head)]
            
            # For each vertex u of G: Run Dijkstra's algorithm in G, with edge lengths c(e)'
            # with source vertex u, to compute the shortest-path distance d(u, v)' for each v in G.
            for tail in range(1, self.node_count + 1):
                self.distance = {i: math.inf for i in range(1, self.node_count + 1)}
                self.dijkstra(tail)
                self.shortest_path[tail] = self.distance.copy()
            
            # For each pair u, v in G, return the shortest-path distance d(u, v) = d(u, v)' - p(u) + p(v)
            for vertex, values in self.shortest_path.items():
                for head, distance in values.items():
                    if vertex == head:
                        self.shortest_path[vertex][head] = 0
                    else:
                        if self.shortest_path[vertex][head] != math.inf:
                            self.shortest_path[vertex][head] = self.shortest_path[vertex][head] - self.path[("current", vertex)] + self.path[("current", head)]
                            if self.shortest > self.shortest_path[vertex][head]:
                                self.shortest = self.shortest_path[vertex][head]
            
            return self.shortest
            
                
        

if __name__ == "__main__":  
    import time
    
    start = time.time()
    
    filename1 = "g1.txt"
    vertice_count1, edge_count1, nodes1, reversed_nodes1 = load(filename1)
    graph1 = Graph(nodes1, reversed_nodes1, vertice_count1, edge_count1)
    print("The shortest path of all-pairs shortest-path of g1 is:", graph1.johnson())
    
    end = time.time()
    print(f"The run time of Johnson Algorithm on g1.txt is {end-start} second(s)")
    print()

    start = time.time()
    
    filename2 = "g2.txt"
    vertice_count2, edge_count2, nodes2, reversed_nodes2 = load(filename2)
    graph2 = Graph(nodes2, reversed_nodes2, vertice_count2, edge_count2)
    print("The shortest path of all-pairs shortest-path of g2 is:", graph2.johnson())
    
    end = time.time()
    print(f"The run time of Johnson Algorithm on g2.txt is {end-start} second(s)")
    print()
    
    
    start = time.time()
    
    filename3 = "g3.txt"
    vertice_count3, edge_count3, nodes3, reversed_nodes3 = load(filename3)
    graph3 = Graph(nodes3, reversed_nodes3, vertice_count3, edge_count3)
    print("The shortest path of all-pairs shortest-path of g3 is:", graph3.johnson())

    end = time.time()
    print(f"The run time of Johnson Algorithm on g3.txt is {end-start} second(s)")
    print()
    
       
        

The shortest path of all-pairs shortest-path of g1 is: There is Negative Cost Cycle!!
The run time of Johnson Algorithm on g1.txt is 583.617301940918 second(s)

The shortest path of all-pairs shortest-path of g2 is: There is Negative Cost Cycle!!
The run time of Johnson Algorithm on g2.txt is 576.8927834033966 second(s)

The shortest path of all-pairs shortest-path of g3 is: -19
The run time of Johnson Algorithm on g3.txt is 549.4141175746918 second(s)

